In [1]:
from pulp import LpMaximize, LpProblem, LpVariable, LpMinimize, LpMaximize, LpStatus, lpSum, listSolvers, value, PULP_CBC_CMD, GLPK_CMD

In [2]:
def CheckForRedundantContraints(model):
    full_objective = value(model.objective)
    redundant_constraints = []
    original_constraints = list(model.constraints.keys())  # constraint names

    for constraint_name in original_constraints:
        removed_constraint = model.constraints.pop(constraint_name)  # Remove by name
        model.solve(PULP_CBC_CMD())  # Re-solve after removal
        new_objective = value(model.objective)

        if new_objective == full_objective:  # If objective remains unchanged, constraint is redundant
            redundant_constraints.append(constraint_name)

        # Restore constraint
        model.constraints[constraint_name] = removed_constraint

    if redundant_constraints:
        print("\nRedundant Constraints Found:")
        for rc in redundant_constraints:
            print(f"- {rc}")
    else:
        print("No redundant constraints detected.")

def ShowConstraints(model):
    original_constraints = list(model.constraints.keys())  # constraint names
    for constraint_name in original_constraints:
        constraint = model.constraints[constraint_name]
        print(constraint)    

def ShowVariables(model):
    for var in model.variables():
        print(f"{var.name} = {var.varValue}")   

In [3]:
# Toy Linear Problem to check PulP solvers

supported_solvers=listSolvers()
print('Supported solvers :')
print(supported_solvers)

solvers_to_test = [PULP_CBC_CMD, GLPK_CMD]

prob = LpProblem("Example", LpMaximize)
x = LpVariable("x", lowBound=0)
y = LpVariable("y", lowBound=0)

# Objective function
prob += 5*x + 7*y

# Constraints (modify to test feasibility)
prob += 2*x + 7*y <= 100
prob += 4*x + 2*y <= 120

for solver in solvers_to_test:
    # Solve the problem
    print(f"Using {solver} to solve problem`")
    prob.solve(solver())

    # Results
    print(f"Solution is {LpStatus[prob.status]}")
    for var in prob.variables():
        print(f"{var.name} = {var.varValue}")

    print(f"{prob.objective.value} = {prob.objective.value()}")

Supported solvers :
['CyLP', 'GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD', 'SAS94', 'SASCAS']
Using <class 'pulp.apis.coin_api.PULP_CBC_CMD'> to solve problem`
Solution is Optimal
x = 26.666667
y = 6.6666667
<bound method LpAffineExpression.value of 5*x + 7*y + 0> = 180.0000019
Using <class 'pulp.apis.glpk_api.GLPK_CMD'> to solve problem`
Solution is Optimal
x = 26.6667
y = 6.66667
<bound method LpAffineExpression.value of 5*x + 7*y + 0> = 180.00018999999998


In [4]:
# Simple integer programming problem - testing PulP's accuracy

# Company manufactures sporks (spoon-fork-knife combo), packets and school packs. 
# Packets are sets of a spork, a napkin and a straw.
# School packs are boxes of 100 packs with additional 10 sporks.
#
# For 1000 sporks, it takes :
# 1. 0.8 hours of molding machine time.
# 2. 0.2 hours of supervisory time
# 3. $2.50 in direct costs.
#
# For 1000 packets with 1 spork, 1 napkin and 1 straw, it takes :
# 1. 1.5 hours of packaging capacity.
# 2. 0.5 hours of supervisory time.
# 3. $4.00 in direct cost.
#
# For 1000 school packs:
# 2.5 hours of packaging capacity.
# 0.5 hours of supervisory time.
# 10 sporks, 100 packets and $8.00 direct costs.

# Any of the three products may be sold in unlimited quantities at prices of $5.00, $15.00, and $300.00 per
# thousand, respectively. If there are 200 hours of production time in the coming month, what products, and
# how much of each, should be manufactured to yield the most profit?

model = LpProblem(name="simple-mixed-integer-prob", sense=LpMaximize)

# Define variables
# x1 = thousands of sporks produced
# x2 = housands of packets  produced
# x3 = thousands of school packs produced

# y1 = x1 - x2 - 10*x3    # 1 packet needs 1 spork and 1 school pack needs 10 sporks
# y2 = x2 - 100*x3        # 100 packets in school packs 
# y3 = x3

# Express x's in terms of y's
# x1 = y1 + y2 + 110*y3
# x2 = y2 + 100*y3
# x3 = y3

# Total profit = 5.00 * y1 + 15.00 * y2 + 300.00 * y3 - (2.5 * x1 + 4.00 * x2 + 8.00 * x3)
# In terms of x's only, Total Profits = 2.5 * x1 + 6 * x2 -1258 * x3

# Define variables
x1 = LpVariable(name="sporks_prod", lowBound=0, cat="Integer")           # thousands of sporks produced
x2 = LpVariable(name="packets_prod", lowBound=0, cat="Integer")          # thousands of packets  produced
x3 = LpVariable(name="school_packs_prod", lowBound=0, cat="Integer")     # thousands of school packs produced


# Objective function
model += 2.5 * x1 + 6 * x2 -1258 * x3, "Profit"

# Constraints due to 200 hours limit for 1 month 
model += 0.8 * x1 <= 200, "Molding"                                 # Sporks requires 0.8 hours of molding per thousand
model += 1.5 * x2 + 2.5 * x3 <= 200, "Packaging"                    # Packets and School packs require 1.5 and 2.5 hours of packaging per thousand
model += 0.2 * x1 + 0.5 * x2 + 0.5 * x3 <= 200, "Supervisory"       # Supervisory hours required for all per thousand

# Solve the problem
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

print("Solver Results :")
print(f"Solution is : {LpStatus[model.status]}")

# for var in model.variables():
#     print(f"Variable {var.name} = {var.varValue}")

print(f"Optimal Production of A: {x1.varValue}")
print(f"Optimal Production of B: {x2.varValue}")
print(f"Optimal Production of C: {x3.varValue}")
print(f"Maximum Profit: {model.objective.value()}")
print(f"Maximum Profit (Manual calcs): {2.5 * x1.varValue + 6 * x2.varValue -1258 * x3.varValue}")


Solver Results :
Solution is : Optimal
Optimal Production of A: 250.0
Optimal Production of B: 133.0
Optimal Production of C: 0.0
Maximum Profit: 1423.0
Maximum Profit (Manual calcs): 1423.0


In [5]:
# Assign truck using integer programming - using direct truck assigment - i.e. each truck is selected using a binary decision variable 
# Major disadvantages - Assigns only 1 truck per time step (such as at a dump event), even if a shovel could use more than 1 for the next planning period ( which need to be estimated)
# **Note** This example does not represent a realistic fleet configuration, too few trucks, so only for showing concept.

# Define the number of trucks and shovels
m = 6  # Number of trucks
n = 4  # Number of shovels

# Create the decision variables X_ij (binary)
X = [[LpVariable(f"X_{i}_{j}", cat="Binary") for j in range(n)] for i in range(m)]

# Define expected travel time parameters W_ij as fixed constants (in minutes)
# Could add expected truck queue time when truck is expected to arrive at nominated shovel
idle_time = [[18, 15, 20, 2],   # Idle time for truck 1 at different shovels
            [18, 45, 18, 32],  # Idle time for truck 2 at different shovels
            [17, 14, 25, 52],  # Idle time for truck 3 at different shovels
            [19, 28, 19, 21],  # Idle time for truck 4 at different shovels
            [16, 10, 17, 33],  # Idle time for truck 5 at different shovels
            [19, 10, 17, 29]]  

# Define the optimization problem
model = LpProblem("direct_truck_assignment_by_travel_minimization", LpMinimize)

# Define the objective function: Minimize total travel time
model += lpSum(idle_time[i][j] * X[i][j] for i in range(m) for j in range(n))

# Constraints (example: each truck is can be assigned to at most one shovel)
for i in range(m):
    model += lpSum(X[i][j] for j in range(n)) <= 1  # Each truck assigned to at most one shovel


for j in range(n):
    model += lpSum(X[i][j] for i in range(m)) == 1 # Each shovel is assigned exactly 1 truck

# Solve the problem
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Print results
for i in range(m):
    for j in range(n):
        if X[i][j].varValue == 1:
            print(f"Truck {i+1}, Shovel {j+1}: Assigned = {X[i][j].varValue}, Idle Time = {idle_time[i][j]} secs ")

print(f"Solution Status: {LpStatus[prob.status]}")
print(f"Minimum total travelling time = {model.objective.value()} secs")        


Truck 1, Shovel 4: Assigned = 1.0, Idle Time = 2 secs 
Truck 2, Shovel 3: Assigned = 1.0, Idle Time = 18 secs 
Truck 5, Shovel 1: Assigned = 1.0, Idle Time = 16 secs 
Truck 6, Shovel 2: Assigned = 1.0, Idle Time = 10 secs 
Solution Status: Optimal
Minimum total travelling time = 46.0 secs


$$
\begin{array}{l l}
\text{General tranportation (Temeng) optimization of material flow rates from shovels to dumps, crushers, stockpiles.} \\
\text{In this basic formulation, there is no distiguishing between dumps, crushers or stockpiles, only dumps.} \\
\text{The solution of this can provide the required material flow as input to the real time dispatch optimizer.} \\
\text{for example, the flow rates are the references for computing deviations from production scheduled levels} \\
\quad \\
\text{m - number of supply points - such as shovels}  \\
\quad \\
\text{n - number of  demand points -  such as dumps} \\
\quad \\
\text{max/min} \quad \sum_{i=1}^{n} \sum_{j=1}^{m} C_{ij} X_{ij} \\
\quad \\
\text{subject to :} \\
\quad \\
\sum_{j=1}^{m} X_{ij} \leq s_{i} & \text{i=1,..,n} & \quad \text{Supply constraints }\\ 
\quad \\
\sum_{i=1}^{n} X_{ij} \geq d_{i} & \text{j=1,..,m} & \quad \text{Demand constraints }\\ 
\quad \\
X_{ij} \geq 0 &  \text{i=1,..,n} & \text{j=1,..,m}\\
\quad \\
\sum_{i=1}^{m} s_{i} \geq \sum_{i=1}^{n} d_{i} & \text{Ensure that demand can be met} \\
\quad\\
\text{where,} \\
\quad\\
C_{ij} - \text{Cost of transporting material from supply i to demand j} \\
\quad \\
X_{ij} - \text{number of units shipped from supply point i to demand point j }
\end{array}
$$

In [ ]:
# An implementation of the transportation model

# Define the number of trucks and shovels
m = 6  # Number of mining faces assigned to shovels - these are the sources of material   
n = 3  # Number of dumping points # crushers, stockpiles, waste dumps - these are the sinks of materials

# Create the decision variables X_ij 
X = [[LpVariable(f"X_{i}_{j}", lowBound=0, cat="Continuous") for j in range(n)] for i in range(m)]  # Tons of material transported from shovels to dumps

# Define cost parameters C_ij as fixed constants (but can change as conditions changes, such as speed limits). 
# For this example, we use travelling duration as the cost
# Each row is one of m sources and each column is one of n sinks 
# Each element in the matrix is cost of fuel per ton of material transported between a source and a sink       
# 
litres_per_hour_ton = 0.5 # litres per hour per ton GVW
empty_weight = 200 # tons

dump_capacities = [2000, 1800, 22000]

# haulage costs = travel times in minutes * 
travel_durations = [[22, 18, 20],    # shovel 1 to each of n dumping points
                    [24, 40, 38],    # shovel 2 to each of n dumping points
                    [17, 14, 25],    # shovel 3 to each of n dumping points
                    [19, 28, 23],    # shovel 4 to each of n dumping points
                    [16, 36, 26],    # shovel 5 to each of n dumping points
                    [33, 35, 21]]    # shovel 6 to each of n dumping points

shift_target_tonnage = 6000

litres_per_ton = [[d * litres_per_hour_ton for d in row] for row in travel_durations]
print(litres_per_ton)

# Define the optimization problem
model = LpProblem("load_to_dump_tons_to_transport", LpMinimize)

# Define the objective function: Minimize total 
model += lpSum(litres_per_ton[i][j] * (X[i][j] + empty_weight) for i in range(m) for j in range(n))

# Limit capacity from each mining face
for i in range(m):
    model += lpSum(X[i][j] for j in range(n)) <= 2000
    model += lpSum(X[i][j] for j in range(n)) >= 200

model += lpSum(X[i][j] for i in range(m) for j in range (n)) >= shift_target_tonnage

# Solve the problem
model.solve()

# Print results
for i in range(m):
    for j in range(n):
        if X[i][j].varValue > 0:
            print(f"Shovel {i+1}, Dump {j+1}: Tonnage = {X[i][j].varValue}, litres = {X[i][j].varValue * litres_per_ton[i][j]}")

print(f"Solution Status: {LpStatus[model.status]}")
print(f"Minimum total fuel use = {model.objective.value()} litres")        

ShowConstraints(model)


[[11.0, 9.0, 10.0], [12.0, 20.0, 19.0], [8.5, 7.0, 12.5], [9.5, 14.0, 11.5], [8.0, 18.0, 13.0], [16.5, 17.5, 10.5]]
Shovel 1, Dump 2: Tonnage = 1400.0, litres = 12600.0
Shovel 2, Dump 1: Tonnage = 200.0, litres = 2400.0
Shovel 3, Dump 2: Tonnage = 2000.0, litres = 14000.0
Shovel 4, Dump 1: Tonnage = 200.0, litres = 1900.0
Shovel 5, Dump 1: Tonnage = 2000.0, litres = 16000.0
Shovel 6, Dump 3: Tonnage = 200.0, litres = 2100.0
Solution Status: Optimal
Minimum total fuel use = 94500.0 litres
X_0_0 + X_0_1 + X_0_2 <= 2000.0
X_0_0 + X_0_1 + X_0_2 >= 200.0
X_1_0 + X_1_1 + X_1_2 <= 2000.0
X_1_0 + X_1_1 + X_1_2 >= 200.0
X_2_0 + X_2_1 + X_2_2 <= 2000.0
X_2_0 + X_2_1 + X_2_2 >= 200.0
X_3_0 + X_3_1 + X_3_2 <= 2000.0
X_3_0 + X_3_1 + X_3_2 >= 200.0
X_4_0 + X_4_1 + X_4_2 <= 2000.0
X_4_0 + X_4_1 + X_4_2 >= 200.0
X_5_0 + X_5_1 + X_5_2 <= 2000.0
X_5_0 + X_5_1 + X_5_2 >= 200.0
X_0_0 + X_0_1 + X_0_2 + X_1_0 + X_1_1 + X_1_2 + X_2_0 + X_2_1 + X_2_2 + X_3_0 + X_3_1 + X_3_2 + X_4_0 + X_4_1 + X_4_2 + X_5_0 + X

In [ ]:
# Direct assignment
m = 6  # Number of mining faces assigned to shovels - these are the sources of material   
n = 3  # Number of dumping points # crushers, stockpiles, waste dumps - these are the sinks of materials

# Create the decision variables X_ij (binary)
X = [[LpVariable(f"X_{i}_{j}", cat="Binary") for j in range(n)] for i in range(m)]  # Tons of material transported from shovels to dumps

# Define cost parameters C_ij as fixed constants (but can change as conditions changes, such as speed limits). 
# For this example, we use travelling duration as the cost
# Each row is one of m sources and each column is one of n sinks 
# Each element in the matrix is travelling time between a source and a sink        
C = [[22, 18, 20],    # Traveling time from shovel 1 to each of n dumping points
     [18, 40, 18],    # Traveling time from shovel 2 to each of n dumping points
     [17, 14, 25],    # Traveling time from shovel 3 to each of n dumping points
     [19, 28, 19],    # Traveling time from shovel 4 to each of n dumping points
     [16, 10, 17],    # Traveling time from shovel 5 to each of n dumping points
     [19, 10, 17]]    # Traveling time from shovel 6 to each of n dumping points

# Define the optimization problem
model = LpProblem("Load_to_Dump_flow_rates", LpMinimize)

# Define the objective function: Minimize total 
model += lpSum(C[i][j] * X[i][j] for i in range(m) for j in range(n))

# Constraints (example: each truck is can be assigned to at most one shovel)
for i in range(m):
    model += lpSum(X[i][j] for j in range(n)) <= 1  # Each truck assigned to at most one shovel


for j in range(n):
    model += lpSum(X[i][j] for i in range(m)) == 1 # Each shovel is assigned exactly 1 truck

# Solve the problem
model.solve()

# Print results
for i in range(m):
    for j in range(n):
        if X[i][j].varValue == 1:
            print(f"Truck {i+1}, Shovel {j+1}: Assigned = {X[i][j].varValue}, Travel Time = {W[i][j]} min")

print(f"Solution Status: {LpStatus[model.status]}")
print(f"Minimum total idle time = {model.objective.value()} mins")        


$$
\begin{array}{l l}
\text{Tires are rated with a maximum Tkph = } (\frac{{Q_loaded} + Q_{empty}}{2})(\frac{N L}{H}) \\
\quad\\
where,
\quad\\
Q_{loaded} \text{= load per tire when truck is loaded} \\
Q_{empty} \text{= load per tire when truck is empty} \\
\text{N = Number of cycles per working day} \\
\text{𝐿 = Length per cycle (round trip) in kilometers} \\
\text{𝐻 = Number of operating hours per day} \\
\quad\\
\text{Tkph as a product of load(tons) and vehicle speed(kph) is sometimes used as a measure of haulage productivity} \\
\text{}
\text{Loads and travelling speeds are somewhat inversely related, discounting the effects of other factors such as road resistance.} \\
\text{Road resistance can be factored in as addition (e,g. positive grade) or subtraction (e.g. negative grade) from the load.}
\end{array}
$$

In [ ]:
# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2']
dumps = ['Dump1', 'Dump2']

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 200, 'Shovel2': 250}

# Dumping location capacity (tons per hour)
cap = {'Dump1': 180, 'Dump2': 270}

# Travel duration cost (minutes per ton)
travel_time = {
    ('Shovel1', 'Dump1'): 25,
    ('Shovel1', 'Dump2'): 28,
    ('Shovel2', 'Dump1'): 27,
    ('Shovel2', 'Dump2'): 26
}

# Shift duration (hours)
T_shift = 8

# Total hauled tonnage for the shift
total_tonnage = 3000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in dumps }

# Objective function: Minimize total travel duration
model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in dumps)

# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in dumps) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in dumps:
    model += lpSum(x[i, j] for i in shovels) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in dumps) * T_shift == total_tonnage, "TotalTonnageConstraint"

# Solve the model
model.solve()

# Print results
for i in shovels:
    for j in dumps:
        print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")

print(f"Solution Status: {LpStatus[prob.status]}")
print(f"Total transportation cost: {model.objective.value()}")


In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 12000, 'Shovel2': 250, 'Shovel3': 220, 'Shovel4': 180, 'Shovel5': 210, 'Shovel6': 230, 'Shovel7': 260}  # Shovel7 mines waste

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (total minutes)
travel_time = {
    ('Shovel1', 'Crusher1'): 15, ('Shovel1', 'Crusher2'): 18, ('Shovel1', 'Stockpile1'): 25, ('Shovel1', 'Stockpile2'): 28, ('Shovel1', 'Stockpile3'): 30,
    ('Shovel2', 'Crusher1'): 17, ('Shovel2', 'Crusher2'): 20, ('Shovel2', 'Stockpile1'): 27, ('Shovel2', 'Stockpile2'): 29, ('Shovel2', 'Stockpile3'): 32,
    ('Shovel3', 'Crusher1'): 19, ('Shovel3', 'Crusher2'): 16, ('Shovel3', 'Stockpile1'): 26, ('Shovel3', 'Stockpile2'): 30, ('Shovel3', 'Stockpile3'): 34,
    ('Shovel4', 'Crusher1'): 18, ('Shovel4', 'Crusher2'): 22, ('Shovel4', 'Stockpile1'): 28, ('Shovel4', 'Stockpile2'): 30, ('Shovel4', 'Stockpile3'): 36,
    ('Shovel5', 'Crusher1'): 15, ('Shovel5', 'Crusher2'): 18, ('Shovel5', 'Stockpile1'): 25, ('Shovel5', 'Stockpile2'): 27, ('Shovel5', 'Stockpile3'): 30,
    ('Shovel6', 'Crusher1'): 17, ('Shovel6', 'Crusher2'): 20, ('Shovel6', 'Stockpile1'): 27, ('Shovel6', 'Stockpile2'): 29, ('Shovel6', 'Stockpile3'): 32,
    ('Shovel7', 'WasteDump'): 12  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 60000  # Large-scale test case - each crusher at 2000 tons/hour for 10 hours shift = 40000 tons. Remainder goes to stock piles (2 iron ore)

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time }

# Objective function: Minimize total travel duration + crusher balance penalty
balance_diff = LpVariable("BalanceDiff", lowBound=0)
model += lpSum(travel_time[i, j] * x[i,j] for i in shovels for j in all_dumps if (i, j) in travel_time) + balance_diff

# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage, "TotalTonnageConstraint"

# Waste material constraint: Any material from Shovel7 must only go to WasteDump
model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Crusher prioritization: Ensure crushers receive more ore before stockpiles
for j in stockpiles:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= 0.1 * (cap["Crusher1"] + cap["Crusher2"]), f"CrusherPriority_{j}"

model += balance_diff >= lpSum(x[i, "Crusher1"] for i in shovels if (i, "Crusher1") in x) - lpSum(x[i, "Crusher2"] for i in shovels if (i, "Crusher2") in x)


# Crushers must be balanced as evenly as possible
# model += balance_diff >= lpSum(x[i, "Crusher1"] for i in shovels) - lpSum(x[i, "Crusher2"] for i in shovels)
# model += balance_diff >= lpSum(x.get((i, "Crusher1"), 0) for i in shovels) - lpSum(x.get((i, "Crusher2"), 0) for i in shovels)

#model += balance_diff >= lpSum(x[i, "Crusher2"] for i in shovels) - lpSum(x[i, "Crusher1"] for i in shovels)

# Solve the model with PuLP CBC solver for better precision
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Print results
for i in shovels:
    for j in all_dumps:
        if (i, j) in travel_time:
            print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")
print(f"Solution Status: {LpStatus[prob.status]}")
print(f"Total transportation cost: {model.objective.value()}")

# Test for redundant constraints



$$
\begin{array}{l l}
\text{Minimize idle times approach} \\
\quad \\
\text{minimize} \quad \sum_{i=1}^{T} \sum_{j=1}^{D} \sum_{j=1}^{S} C_{tds} X_{tds} + VBN(1-AF) \\
\quad \\
\text{subject to,} \\
\quad \\
\sum_{d=1}^{D} \sum_{s=1}^{S} tc_t * X_{tds} \leq TC_{t} & \text{t=1,..,T} & \quad \text{Truck t cannot accept loads more than its nominal capacity}\\ 
\quad \\
\sum_{t=1}^{T} \sum_{d=1}^{D} tc_t * X_{tds} \leq sc_{s} & \text{s=1,..,S} & \quad \text{ensures that summation of nominal capacity of all the trucks assigned to shovels does not exceed the shovel’s nominal digging rate (capacity)}\\ 
\quad \\
\sum_{t=1}^{T} \sum_{s=1}^{S} tc_t * X_{tds} \geq AF * pc_{d} & \text{d=1,..,D} \\ 
\quad \\
X_{tds} \in \text{\{}{0,1}\text{\}}&\quad\text{Binary decision varriables} \\
\quad \\
mf = \sum_{t=1}^{T} tc_t * X_{tds} / \sum_{s=1}^{S} \sum_{d=1}^{D} (pf_{sd} - pmsf_{sd}) \quad \\
\quad \\
C_{tds}= ltt_{td} + qt_{td} + dt_{td} + ett_{ts} - \sum_{tt=1}^{TT}(tinq_{tts} + tenr_{tts} x (st_{tts} + lt_{tts}) \\
\quad \\
where, \quad \\
\quad \\
X_{tds} \in \text{\{}{0,1}\text{\}}&\quad\text{Binary decision varriables} \\
\quad \\

\end{array}
$$

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus, value, PULP_CBC_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 12000, 'Shovel2': 250, 'Shovel3': 220, 'Shovel4': 180, 'Shovel5': 210, 'Shovel6': 230, 'Shovel7': 260}

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (minutes per ton)
travel_time = {
    ('Shovel1', 'Crusher1'): 15, ('Shovel1', 'Crusher2'): 18, ('Shovel1', 'Stockpile1'): 25,
    ('Shovel2', 'Crusher1'): 17, ('Shovel2', 'Crusher2'): 20, ('Shovel2', 'Stockpile1'): 27,
    ('Shovel3', 'Crusher1'): 19, ('Shovel3', 'Crusher2'): 16, ('Shovel3', 'Stockpile1'): 26,
    ('Shovel4', 'Crusher1'): 18, ('Shovel4', 'Crusher2'): 22, ('Shovel4', 'Stockpile1'): 28,
    ('Shovel5', 'Crusher1'): 15, ('Shovel5', 'Crusher2'): 18, ('Shovel5', 'Stockpile1'): 25,
    ('Shovel6', 'Crusher1'): 17, ('Shovel6', 'Crusher2'): 20, ('Shovel6', 'Stockpile1'): 27,
    ('Shovel7', 'WasteDump'): 12  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 60000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = {(i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time}

# Objective function: Minimize total travel duration
model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time)

# Constraints (Named for easier removal & restoration)
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage, "TotalTonnageConstraint"

model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Solve the full problem
model.solve(PULP_CBC_CMD())
full_objective = value(model.objective)

print(f"Original Objective Value: {full_objective}")
print(f"Solution Status: {LpStatus[model.status]}")

# **Step 2: Check Redundancy by Removing Constraints One at a Time**
redundant_constraints = []
original_constraints = list(model.constraints.keys())  # Store constraint names

for constraint_name in original_constraints:
    removed_constraint = model.constraints.pop(constraint_name)  # Remove by name
    model.solve(PULP_CBC_CMD())  # Re-solve after removal
    new_objective = value(model.objective)

    if new_objective == full_objective:  # If objective remains unchanged, constraint is redundant
        redundant_constraints.append(constraint_name)

    # Restore constraint
    model.constraints[constraint_name] = removed_constraint

# **Step 3: Report Redundant Constraints**
print("\nRedundant Constraints Found:")
if redundant_constraints:
    for rc in redundant_constraints:
        print(f"- {rc}")
else:
    print("No redundant constraints detected.")


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Example input data
shovels = ["Shovel_1", "Shovel_2", "Shovel_3"]
trucks = ["Truck_A", "Truck_B", "Truck_C", "Truck_D", "Truck_E"]
tonnage_target = {"Shovel_1": 500, "Shovel_2": 600, "Shovel_3": 550}
actual_tonnage = {"Shovel_1": 400, "Shovel_2": 620, "Shovel_3": 500}
truck_capacity = 50  # Each truck carries 50 tons
waiting_times = {
    ("Truck_A", "Shovel_1"): 5,
    ("Truck_B", "Shovel_1"): 3,
    ("Truck_C", "Shovel_2"): 4,
    ("Truck_D", "Shovel_2"): 6,
    ("Truck_E", "Shovel_3"): 2
}

# Compute tonnage deviation for needy shovels
tonnage_deviation = {s: max(0, tonnage_target[s] - actual_tonnage[s]) for s in shovels}

# Compute required trucks per shovel
required_trucks = {s: tonnage_deviation[s] // truck_capacity for s in shovels}

# Initialize optimization problem
prob = LpProblem("Truck_Dispatch_Optimization", LpMinimize)

# Decision variables: x[i, j] = 1 if truck i is assigned to shovel j, else 0
x = {(t, s): LpVariable(f"x_{t}_{s}", 0, 1, cat="Binary") for t in trucks for s in shovels}

# Objective: Minimize total truck waiting time at needy shovels
prob += lpSum(x[t, s] * waiting_times.get((t, s), 0) for t in trucks for s in shovels)

# Constraint: Ensure each shovel gets the required number of trucks
for s in shovels:
    prob += lpSum(x[t, s] for t in trucks) == required_trucks[s], f"Shovel_{s}_Truck_Assignment"

# Constraint: Ensure each truck is assigned to only one shovel
for t in trucks:
    prob += lpSum(x[t, s] for s in shovels) <= 1, f"Truck_{t}_Single_Assignment"

# Solve the optimization problem
prob.solve()

# Display results
assigned_trucks = {s: [] for s in shovels}
for t in trucks:
    for s in shovels:
        if x[t, s].varValue == 1:
            assigned_trucks[s].append(t)

# Output results
for s in shovels:
    print(f"{s}: Assigned Trucks -> {assigned_trucks[s]}")


In [ ]:
import pulp

# Create the optimization problem
model = pulp.LpProblem("Dump_Truck_Operator_Training", pulp.LpMaximize)

# Define operators and competencies
operators = ["Op1", "Op2", "Op3"]  # Example operator IDs
competencies = ["Safe_Operation", "Optimal_Braking", "Optimal_Transmission"]
quartiles = [1, 2, 3]

# Training cost per hour
training_cost_per_hour = 100  # Example cost
max_training_hours = 2.5  # Maximum allowed session per operator

# Returns based on quartile improvements
returns = {1: 20, 2: 18, 3: 15}
training_time = {1: 2, 2: 1, 3: 0.5}  # Hours required for each quartile improvement

# Decision Variables
x = pulp.LpVariable.dicts("Training_Hours", (operators, competencies), lowBound=0, upBound=max_training_hours, cat="Continuous")
y = pulp.LpVariable.dicts("Quartile_Improvement", (operators, competencies, quartiles), cat="Binary")
z = pulp.LpVariable.dicts("Training_Assigned", operators, cat="Binary")

# Objective Function: Maximize Net Benefit (Returns - Training Cost)
model += pulp.lpSum(returns[q] * y[o][c][q] for o in operators for c in competencies for q in quartiles) - \
         pulp.lpSum(training_cost_per_hour * x[o][c] for o in operators for c in competencies)

# Constraints
for o in operators:
    for c in competencies:
        # Ensure training hours match the required quartile improvements
        model += x[o][c] == pulp.lpSum(training_time[q] * y[o][c][q] for q in quartiles)
        
        # Training hours must be within limit
        model += x[o][c] <= max_training_hours * z[o]

        # Sequential quartile progression constraint
        for q in quartiles[1:]:  # Start from 2nd quartile
            model += y[o][c][q] <= y[o][c][q-1]

# Solve the model
model.solve()

# Print results
print("Training Plan:")
for o in operators:
    for c in competencies:
        print(f"Operator {o}, Competency {c}, Training Hours: {x[o][c].varValue}")

print(f"Total Objective Value (Net Benefit): {pulp.value(model.objective)}")


In [ ]:
x